In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
pip install pandas


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of initial URLs (seeds) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://www.worldpoliticsreview.com/",
    "https://www.cnbc.com/politics/",
    "https://www.politico.com/",
    "https://edition.cnn.com/politics",
    "https://www.politico.eu/"
]
def extract_publication_date(url):
    # Extract the publication date from the URL (assuming it's in yyyy-mm-dd format)
    parts = url.split("/")
    for part in parts:
        if len(part) == 10 and part.count("-") == 2:
            return part
    return "0000-00-00"  # Return a default date value

# Rest of your code remains the same

# Function to crawl a URL
def crawl(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            # Now you can use BeautifulSoup to extract information from the page
            # For example, extracting links:
            links = [link.get("href") for link in soup.find_all("a")]
            return links
        else:
            print(f"Failed to fetch {url}")
            return []
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")
        return []

def main():
    all_links = []
    for url in initial_urls:
        links = crawl(url)
        all_links.extend(links)
    
    # Create a DataFrame from the links
    df = pd.DataFrame({"Links": all_links})
    
    # Extract publication dates and sort by date
    df["PublicationDate"] = df["Links"].apply(extract_publication_date)
    df = df.dropna(subset=["PublicationDate"])  # Remove rows with no date
    df = df.sort_values(by="PublicationDate")
    
    df.to_csv("sorted_crawled_links.csv", index=False)

if __name__ == "__main__":
    main()






AttributeError: 'NoneType' object has no attribute 'split'

In [4]:
import pandas as pd
df = pd.read_csv('crawled_links.csv')
df

,Links
0,#navigation
1,#search
2,#main
3,#footer
4,https://foreignpolicy.com/
...,...
1918,/frequently-asked-questions/
1919,https://www.politico.eu/registration/
1920,https://www.politico.eu/wp-login.php?action=lo...
1921,https://www.politico.eu/privacy-policy/


In [11]:
from IPython.display import display, HTML
import pandas as pd

# Assuming you have a DataFrame named df with a "Links" column

for link_url in df["Links"]:
    
    display(HTML(f'<a href="{link_url}" target="_blank">{link_url}</a>'))


In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, date

# List of initial URLs (seeds) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://www.worldpoliticsreview.com/",
    "https://www.cnbc.com/politics/",
    "https://www.politico.com/",
    "https://edition.cnn.com/politics",
    "https://www.politico.eu/"
]

# Function to crawl a URL
def crawl(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            # Modify this part to identify the news links and their publication dates
            news_links = []
            for link in soup.find_all("a"):
                href = link.get("href")
                # Filter out non-news links based on your criteria
                if "news" in href:  # Adjust this condition based on the structure of news URLs
                    publication_date = extract_publication_date(link)  # Implement this function
                    if publication_date == date.today():
                        news_links.append(href)
            return news_links
        else:
            print(f"Failed to fetch {url}")
            return []
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")
        return []

# Function to extract publication date from a link
def extract_publication_date(link):
    # Implement this function to extract and parse the publication date
    # from the link element and return it as a date object
    pass

# Main function
def main():
    all_links = []
    for url in initial_urls:
        links = crawl(url)
        all_links.extend(links)
    
    # Create a DataFrame from the links
    df = pd.DataFrame({"Links": all_links})
    df.to_csv("crawled_liinks.csv", index=False)  # Save the DataFrame to a CSV file

if __name__ == "__main__":
    main()


An error occurred while crawling https://edition.cnn.com/politics: argument of type 'NoneType' is not iterable
An error occurred while crawling https://www.politico.eu/: argument of type 'NoneType' is not iterable


In [16]:
import pandas as pd
df = pd.read_csv('crawled_liinks.csv')
df

,Links


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the previously collected links
df_links = pd.read_csv('crawled_links.csv')
article_links = df_links['Links'].tolist()

def parse_article(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")

            # Extract title
            title_element = soup.find("h1")
            title = title_element.text.strip() if title_element else ""

            # Extract text
            paragraphs = soup.find_all("p")
            text = " ".join([p.text.strip() for p in paragraphs])

            # Extract image URL
            image_element = soup.find("img")
            image = image_element.get("src") if image_element else ""

            article_details = {
                "Title": title,
                "Text": text,
                "Image": image,
                "URL": url
            }
            return article_details
        else:
            print(f"Failed to fetch {url}")
            return None
    except Exception as e:
        print(f"An error occurred while parsing details from {url}: {e}")
        return None


# Main function
def main():
    all_links = []
    for url in initial_urls:
        links = crawl(url)
        all_links.extend(links)

    # Create a DataFrame from the links
    df = pd.DataFrame({"Links": all_links})

    # Filter out non-article URLs
    df = df[df["Links"].str.contains("https://|/\d{4}/\d{2}/\d{2}/")]

    # Initialize a list to hold parsed article details
    article_details_list = []

    for url in df["Links"]:
        article_details = parse_article(url)
        if article_details:
            article_details_list.append(article_details)

    # Create a DataFrame from the parsed article details
    df_articles = pd.DataFrame(article_details_list)

    # Save the DataFrame to a CSV file
    df_articles.to_csv("parsed_articles.csv", index=False)

if __name__ == "__main__":
    main()


Failed to fetch https://fpguide.foreignpolicy.com/2023-career/
An error occurred while parsing details from /2023/04/24/brics-currency-end-dollar-dominance-united-states-russia-china/?tpcc=recirc_trending062921: Invalid URL '/2023/04/24/brics-currency-end-dollar-dominance-united-states-russia-china/?tpcc=recirc_trending062921': No scheme supplied. Perhaps you meant http:///2023/04/24/brics-currency-end-dollar-dominance-united-states-russia-china/?tpcc=recirc_trending062921?
An error occurred while parsing details from /2023/08/24/prigozhin-dead-plane-crash-russia-wagner-group-putin-violence/?tpcc=recirc_trending062921: Invalid URL '/2023/08/24/prigozhin-dead-plane-crash-russia-wagner-group-putin-violence/?tpcc=recirc_trending062921': No scheme supplied. Perhaps you meant http:///2023/08/24/prigozhin-dead-plane-crash-russia-wagner-group-putin-violence/?tpcc=recirc_trending062921?
An error occurred while parsing details from /2023/08/24/yevgeny-prigozhin-death-putin-russia-war-ukraine/?t

KeyboardInterrupt: 

# ================================================================

In [1]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of initial URLs (seeds) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://www.worldpoliticsreview.com/",
    "https://www.cnbc.com/politics/",
    "https://www.politico.com/",
    "https://edition.cnn.com/politics",
    "https://www.politico.eu/"
]

# Function to crawl a URL
def crawl(url):
    try:
        if not url.startswith("http"):
            return []  # Skip URLs without a scheme
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            links = [link.get("href") for link in soup.find_all("a") if link.get("href")]
            return links
        else:
            print(f"Failed to fetch {url}")
            return []
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")
        return []

# Function to extract article details from a URL
def extract_article_details(article_url):
    try:
        response = requests.get(article_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract title
            title_element = soup.find("h1")  # Adjust this based on the actual title tag
            title = title_element.get_text() if title_element else None
            
            # Extract date
            date_element = soup.find("time")  # Adjust this based on the actual date tag
            date = date_element["datetime"] if date_element else None
            
            # Extract text
            text_elements = soup.find_all("p")  # Adjust this based on the actual text tag
            text = "\n".join(element.get_text() for element in text_elements)
            
            # Extract images
            image_elements = soup.find_all("img")  # Adjust this based on the actual image tag
            images = [element["src"] for element in image_elements]
            
            return {
                "Title": title,
                "Date": date,
                "Text": text,
                "Images": images,
                "Link": article_url
            }
        else:
            print(f"Failed to fetch {article_url}")
            return None
    except Exception as e:
        print(f"An error occurred while crawling {article_url}: {e}")
        return None

# Main function
def main():
    all_article_details = []
    for url in initial_urls:
        links = crawl(url)
        if links:
            for link in links:
                article_details = extract_article_details(link)
                if article_details:
                    all_article_details.append(article_details)
    
    # Create a DataFrame from the article details
    df = pd.DataFrame(all_article_details)
    df.to_csv("cs2w.csv", index=False)  # Save the DataFrame to a CSV file

if __name__ == "__main__":
    main()


An error occurred while crawling #navigation: Invalid URL '#navigation': No scheme supplied. Perhaps you meant http://#navigation?
An error occurred while crawling #search: Invalid URL '#search': No scheme supplied. Perhaps you meant http://#search?
An error occurred while crawling #main: Invalid URL '#main': No scheme supplied. Perhaps you meant http://#main?
An error occurred while crawling #footer: Invalid URL '#footer': No scheme supplied. Perhaps you meant http://#footer?
An error occurred while crawling https://foreignpolicy.com/: 'src'
An error occurred while crawling #: Invalid URL '#': No scheme supplied. Perhaps you meant http://#?
An error occurred while crawling /gift-subscriptions/?tpcc=navbar_gift: Invalid URL '/gift-subscriptions/?tpcc=navbar_gift': No scheme supplied. Perhaps you meant http:///gift-subscriptions/?tpcc=navbar_gift?
An error occurred while crawling /subscribe: Invalid URL '/subscribe': No scheme supplied. Perhaps you meant http:///subscribe?
An error occu

An error occurred while crawling https://foreignpolicy.com/author/edoardo-campanella/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/john-haigh/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/24/brics-summit-expansion-new-members-china-russia-saudi-arabia-iran-uae-us/?tpcc=recirc_latest062921: 'src'
An error occurred while crawling https://foreignpolicy.com/category/world-brief/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/alexandra-sharp/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/24/republicans-united-states-gop-debate-foreign-policy-ukraine/?tpcc=recirc_latest062921: 'src'
An error occurred while crawling https://foreignpolicy.com/category/news/report/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/jack-detsch/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/24/prigozhin-putin-wagner-group-russia-africa/?tpcc=recirc_lates

An error occurred while crawling https://foreignpolicy.com/author/michael-kugelman/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/23/india-moon-landing-chandrayaan-geopolitics-modi/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/nosmot-gbadamosi/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/23/brics-ukraine-russia-africa-summit-ramaphosa/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/james-palmer/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/22/china-economy-crisis-investors-xi-health-care-property/: 'src'
An error occurred while crawling https://foreignpolicy.com/author/anchal-vohra/: 'src'
An error occurred while crawling https://foreignpolicy.com/2023/08/22/rishi-sunak-tories-economy-technocrat/: 'src'
An error occurred while crawling /the-magazine: Invalid URL '/the-magazine': No scheme supplied. Perhaps you meant http:///the-magazine?
An error 

KeyboardInterrupt: 

In [11]:
import pandas as pd
df33 = pd.read_csv('cs2w.csv')
df33

EmptyDataError: No columns to parse from file

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of dictionaries to store article information
articles = []

# List of initial URLs (seeds) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://www.worldpoliticsreview.com/",
    "https://www.cnbc.com/politics/",
    "https://www.politico.com/",
    "https://edition.cnn.com/politics",
    "https://www.politico.eu/"
]

# Function to crawl a URL
def crawl(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract website name
            website_name = url.split("//")[1].split(".")[0]
            
            # Extract article information
            for article in soup.find_all("article"):
                title_element = article.find("h2")
                text_element = article.find("p")
                link_element = article.find("a")
                image_element = article.find("img")
                
                article_info = {
                    "Title": title_element.text.strip() if title_element else None,
                    "Text": text_element.text.strip() if text_element else None,
                    "Link": link_element["href"] if link_element else None,
                    "Image": image_element["src"] if image_element else None,
                    "Website": website_name,
                    "Date": None  # Placeholder for article date
                }
                articles.append(article_info)
                
                # If there's a link to the full article, try to gather more data from there
                if link_element:
                    article_url = link_element["href"]
                    extract_article_data(article_url, website_name)
        else:
            print(f"Failed to fetch {url}")
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")

# Function to extract more data from an individual article page
def extract_article_data(article_url, website_name):
    try:
        response = requests.get(article_url)
        if response.status_code == 200:
            article_soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract article title
            title_element = article_soup.find("h1", class_="hed-heading")  # Replace "hed" with the correct class
            article_title = title_element.get_text() if title_element else None
            
            # Extract article text
            article_text = ""
            paragraphs = article_soup.find_all("p")
            for paragraph in paragraphs:
                article_text += paragraph.get_text() + "\n"
            
            # Extract article date
            date_element = article_soup.find("time", class_="meta-data")  # Replace "date-time" with the correct class
            article_date = date_element["datetime"] if date_element else None
            
            # Extract article image
            image_element = article_soup.find("image-attachment -ratioscale  horizontal-orientation image-loaded")  # Replace "img" with the correct tag
            article_image = image_element["src"] if image_element else None
            
            # Update the articles list with additional information
            for article in articles:
                if article["Link"] == article_url:
                    article["Title"] = article_title
                    article["Text"] = article_text
                    article["Date"] = article_date
                    article["Image"] = article_image
        else:
            print(f"Failed to fetch {article_url}")
    except Exception as e:
        print(f"An error occurred while crawling {article_url}: {e}")

# Main function
def main():
    for url in initial_urls:
        crawl(url)
    
    # Create a DataFrame from the article information
    df = pd.DataFrame(articles)
    df.to_csv("crawled_articles334.csv", index=False)  # Save the DataFrame to a CSV file

if __name__ == "__main__":
    main()


Failed to fetch https://www.politicopro.com


In [10]:
import pandas as pd
df33 = pd.read_csv('crawled_articles334.csv')
df33

,Title,Text,Link,Image,Website,Date
0,NaN,"RIGA, Latvia—When prosecutors in Berlin confir...",https://www.worldpoliticsreview.com/journalist...,NaN,www,NaN
1,NaN,"RIGA, Latvia—When prosecutors in Berlin confir...",https://www.worldpoliticsreview.com/journalist...,NaN,www,NaN
2,NaN,The war in Ukraine has continued to test natio...,https://www.worldpoliticsreview.com/uae-russia...,NaN,www,NaN
3,NaN,Since Donald Trump’s shock victory in the 2016...,https://www.worldpoliticsreview.com/trump-brex...,NaN,www,NaN
4,NaN,Despite having little to show from his first f...,https://www.worldpoliticsreview.com/zimbabwe-e...,NaN,www,NaN
5,NaN,"As the summer vacation season fizzles out, wor...",https://www.worldpoliticsreview.com/g20-brics-...,NaN,www,NaN
6,NaN,Guatemalan voters turned out in large numbers ...,https://www.worldpoliticsreview.com/guatemala-...,NaN,www,NaN
7,NaN,When Sierra Leone’s new parliament met for its...,https://www.worldpoliticsreview.com/sierra-leo...,NaN,www,NaN
8,NaN,"World Politics Review LLC 401 E. Jackson St, S...",https://www.worldpoliticsreview.com/section/br...,NaN,www,NaN
9,NaN,"World Politics Review LLC 401 E. Jackson St, S...",https://www.worldpoliticsreview.com/section/co...,NaN,www,NaN


In [4]:
styled_df = df3.style.set_properties(**{'text-align': 'center'})
display(styled_df)

,Title,Text,Link,Image,Website
0,BECOME A SUBSCRIBER,Associated Press,https://www.worldpoliticsreview.com/journalists-russia-war-ukraine/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/journalists-russia-war-ukraine-08242023-1.jpg?w=630&h=478&crop=1,www
1,nan,nan,https://www.worldpoliticsreview.com/journalists-russia-war-ukraine/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/journalists-russia-war-ukraine-08242023-1.jpg?w=630&h=478&crop=1,www
2,nan,nan,https://www.worldpoliticsreview.com/uae-russia-us-security/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/uae-russia-us-security-08232023-1.jpg?w=319&h=241&crop=1,www
3,nan,nan,https://www.worldpoliticsreview.com/trump-brexit-populism/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/trump-us-brexit-uk-populism-08232023-1.jpg?w=319&h=241&crop=1,www
4,nan,nan,https://www.worldpoliticsreview.com/zimbabwe-elections-politics-mnangagwa/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/zimbabwe-elections-politics-mnangagwa-08222023-1.jpg?w=319&h=241&crop=1,www
5,nan,nan,https://www.worldpoliticsreview.com/g20-brics-un-summit/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/g20-brics-un-summit-08222023-1.jpg?w=319&h=241&crop=1,www
6,nan,nan,https://www.worldpoliticsreview.com/guatemala-elections-politics-latin-america/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/guatemala-elections-latin-america-populism-08212023-1.jpg?w=519&h=259&crop=1,www
7,nan,nan,https://www.worldpoliticsreview.com/sierra-leone-elections-democracy-bio-slpp-apc-economy/,https://www.worldpoliticsreview.com/wp-content/uploads/2023/08/sierra-leone-elections-democracy-bio-slpp-apc-economy-08212023.jpg?w=519&h=259&crop=1,www
8,nan,nan,https://www.worldpoliticsreview.com/section/briefing/,nan,www
9,nan,nan,https://www.worldpoliticsreview.com/section/column/,nan,www


In [9]:
# Function to extract more data from an individual article page
def extract_article_data(article_url, website_name):
    try:
        response = requests.get(article_url)
        if response.status_code == 200:
            article_soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract additional article information like date
            date_element = article_soup.find("div", class_="update-time")  # Example class, adapt to your needs
            article_date = date_element.text.strip() if date_element else None
            
            # Extract full article text
            article_text = ""
            paragraphs = article_soup.find_all("div", class_="zn-body__paragraph")  # Example class, adapt to your needs
            for paragraph in paragraphs:
                article_text += paragraph.get_text() + "\n"
            
            # Update the articles list with additional information
            for article in articles:
                if article["Link"] == article_url:
                    article["Date"] = article_date
                    article["Text"] = article_text
        else:
            print(f"Failed to fetch {article_url}")
    except Exception as e:
        print(f"An error occurred while crawling {article_url}: {e}")


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of dictionaries to store article information
articles = []

# List of initial URLs (main pages) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://www.worldpoliticsreview.com/",
    # Add other URLs here...
]

# Function to crawl a main page
def crawl_main_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Extract article information and append to articles list
    for article in soup.find_all("article"):
        # Extract title, text, link, image, and website name
        title = article.find("h2").text.strip() if article.find("h2") else None
        text = article.find("p").text.strip() if article.find("p") else None
        link = article.find("a")["href"] if article.find("a") else None
        image = article.find("img")["src"] if article.find("img") else None
        website_name = url.split("//")[1].split(".")[0]
        
        # Create article dictionary and append to articles list
        article_info = {
            "Title": title,
            "Text": text,
            "Link": link,
            "Image": image,
            "Website": website_name
        }
        articles.append(article_info)

# Main function
def main():
    for url in initial_urls:
        crawl_main_page(url)
    
    # Create a DataFrame from the article information
    df = pd.DataFrame(articles)
    df.to_csv("crawled_articl.csv", index=False)  # Save the DataFrame to a CSV file

if __name__ == "__main__":
    main()


In [11]:
import pandas as pd
df33 = pd.read_csv('crawled_articl.csv')
df33

,Title,Text,Link,Image,Website
0,BECOME A SUBSCRIBER,Associated Press,https://www.worldpoliticsreview.com/journalist...,https://www.worldpoliticsreview.com/wp-content...,www
1,NaN,NaN,https://www.worldpoliticsreview.com/journalist...,https://www.worldpoliticsreview.com/wp-content...,www
2,NaN,NaN,https://www.worldpoliticsreview.com/uae-russia...,https://www.worldpoliticsreview.com/wp-content...,www
3,NaN,NaN,https://www.worldpoliticsreview.com/trump-brex...,https://www.worldpoliticsreview.com/wp-content...,www
4,NaN,NaN,https://www.worldpoliticsreview.com/zimbabwe-e...,https://www.worldpoliticsreview.com/wp-content...,www
5,NaN,NaN,https://www.worldpoliticsreview.com/g20-brics-...,https://www.worldpoliticsreview.com/wp-content...,www
6,NaN,NaN,https://www.worldpoliticsreview.com/guatemala-...,https://www.worldpoliticsreview.com/wp-content...,www
7,NaN,NaN,https://www.worldpoliticsreview.com/sierra-leo...,https://www.worldpoliticsreview.com/wp-content...,www
8,NaN,NaN,https://www.worldpoliticsreview.com/section/br...,NaN,www
9,NaN,NaN,https://www.worldpoliticsreview.com/section/co...,NaN,www


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List of dictionaries to store article information
articles = []

# List of initial URLs (seeds) to start crawling from
initial_urls = [
    "https://foreignpolicy.com/",
    "https://edition.cnn.com/politics"
]

# Function to crawl the Foreign Policy website
def crawl_foreign_policy(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract article information based on the Foreign Policy structure
            for article in soup.find_all("article"):
                title_element = article.find("h1", class_="hed")
                text_element = article.find("p")
                date_element = article.find("time", class_="date-time")
                image_element = article.find("img")
                
                article_info = {
                    "Title": title_element.get_text() if title_element else None,
                    "Text": text_element.get_text() if text_element else None,
                    "Date": date_element["datetime"] if date_element else None,
                    "Image": image_element["src"] if image_element else None,
                    "Website": "Foreign Policy",
                    "Link": url
                }
                articles.append(article_info)
        else:
            print(f"Failed to fetch {url}")
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")

# Function to crawl the CNN website
def crawl_cnn(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            # Extract article information based on the CNN structure
            for article in soup.find_all("article"):
                title_element = article.find("span", class_="headline")
                text_element = article.find("div", class_="body")
                date_element = article.find("time", class_="date-time")  # Assuming this class exists
                image_element = article.find("img")
                
                article_info = {
                    "Title": title_element.get_text() if title_element else None,
                    "Text": text_element.get_text() if text_element else None,
                    "Date": date_element["datetime"] if date_element else None,
                    "Image": image_element["src"] if image_element else None,
                    "Website": "CNN",
                    "Link": url
                }
                articles.append(article_info)
        else:
            print(f"Failed to fetch {url}")
    except Exception as e:
        print(f"An error occurred while crawling {url}: {e}")

# Main function
def main():
    for url in initial_urls:
        if "foreignpolicy" in url:
            crawl_foreign_policy(url)
        elif "cnn" in url:
            crawl_cnn(url)
        else:
            print(f"Unsupported website: {url}")
    
    # Create a DataFrame from the article information
    df = pd.DataFrame(articles)
    df.to_csv("crawledasrticles.csv", index=False)

if __name__ == "__main__":
    main()


In [22]:
import pandas as pd
df33 = pd.read_csv('crawledasrticles.csv')
df33

EmptyDataError: No columns to parse from file

# =========================================

# web robot that works

In [11]:
import requests
from bs4 import BeautifulSoup
import csv

# Set up the URL and headers
base_url = "https://news.google.com"
search_term = ['politics','foreign','tunisia']
url = f"{base_url}/search?q={search_term}&hl=en-US&gl=US&ceid=US:en"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Send a GET request to the Google News search results page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find article elements
article_elements = soup.find_all("h3", class_="ipQwMb ekueJc RD0gLb")

# Create a list to store the extracted data
data = []

# Extract article titles and links and store in the data list
for article_element in article_elements:
    article_title = article_element.get_text()
    article_link = base_url + article_element.a["href"][1:]
    data.append([article_title, article_link])

# Define the CSV file name
csv_file_name = "google_news_articles.csv"

# Write the data to the CSV file
with open(csv_file_name, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Title", "Link"])  # Write header
    csv_writer.writerows(data)  # Write data rows

print(f"Data has been saved to {csv_file_name}")


Data has been saved to google_news_articles.csv


In [12]:
import pandas as pd
df33 = pd.read_csv('google_news_articles.csv')
df33

,Title,Link
0,"Analysis: Tunisia isolated, but migration real...",https://news.google.com/articles/CBMibmh0dHBzO...
1,Europe's 'template' for migration: Tunisian st...,https://news.google.com/articles/CBMiU2h0dHBzO...
2,Analysis: Libyan Foreign Minister Dismissed Fo...,https://news.google.com/articles/CBMihAFodHRwc...
3,Tunisia-Libya: closer ties between the two nei...,https://news.google.com/articles/CBMihAFodHRwc...
4,"Tunisian contech startup, SeekMake, gets $539K...",https://news.google.com/articles/CBMiRWh0dHBzO...
...,...,...
94,Tunisia Continues to Face Substantial Financin...,https://news.google.com/articles/CBMicWh0dHBzO...
95,Tunisia’s choice: Migration and realpolitik in...,https://news.google.com/articles/CBMiZGh0dHBzO...
96,Natasha S. Franceschi Deputy Chief of Mission,https://news.google.com/articles/CBMiV2h0dHBzO...
97,Dina Preston Band Brings the Power of Music an...,https://news.google.com/articles/CBMiLmh0dHBzO...


# with dates 

In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import re
# Set up the URL and headers
base_url = "https://news.google.com"
search_term = ['politics','foreign']
url = f"{base_url}/search?q={search_term}&hl=en-US&gl=US&ceid=US:en"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Send a GET request to the Google News search results page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find article elements
article_elements = soup.find_all("h3", class_="ipQwMb ekueJc RD0gLb")

# Create a list to store the extracted data
data = []


# Extract article titles, links, and dates and store in the data list
for article_element in article_elements:
    article_title = article_element.get_text()
    article_link = base_url + article_element.a["href"][1:]
    
    # Extract the publishing date if available
    date_element = article_element.find_next("time")
    if date_element and "datetime" in date_element.attrs:
        article_date = date_element["datetime"][:10]  # Extract the first 10 characters as the date
    else:
        article_date = "No Date Available"
    
    data.append([article_title, article_link, article_date])
# Define the CSV file name
csv_file_name = "political_news_articles2.csv"

# Write the data to the CSV file
with open(csv_file_name, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Title", "Link", "Date"])  # Write header
    csv_writer.writerows(data)  # Write data rows

print(f"Political news data has been saved to {csv_file_name}")


Political news data has been saved to political_news_articles2.csv


In [16]:
import pandas as pd
dfGS = pd.read_csv('political_news_articles2.csv')
dfGS

,Title,Link,Date
0,U.S-China Rivalry Plays Out in Vanuatu's Politics,https://news.google.com/articles/CBMiU2h0dHBzO...,2023-08-28
1,Foreign Ministry Spokesperson Wang Wenbin's Re...,https://news.google.com/articles/CBMiVGh0dHBzO...,2023-08-29
2,Georgetown University Institute of Politics an...,https://news.google.com/articles/CBMipwFodHRwc...,2023-08-28
3,Democrat introduces bill to limit defense cont...,https://news.google.com/articles/CBMiiQFodHRwc...,2023-08-29
4,Libya’s Foreign Minister Najla al-Mangoush dis...,https://news.google.com/articles/CBMic2h0dHBzO...,2023-08-28
...,...,...,...
94,NATO Bid Puts Pressure on Sweden’s Domestic Po...,https://news.google.com/articles/CBMiWWh0dHBzO...,2023-04-20
95,"The DeSantis Foreign Policy: Hard Power, but W...",https://news.google.com/articles/CBMiT2h0dHBzO...,2023-03-22
96,Secretary Antony J. Blinken and Italian Foreig...,https://news.google.com/articles/CBMie2h0dHBzO...,2023-06-12
97,What Does the West Really Know About Xi's China?,https://news.google.com/articles/CBMiT2h0dHBzO...,2023-06-13


In [ ]:
from bs4 import BeautifulSoup
import csv
import re

# Set up the URL and headers
base_url = "https://news.google.com"
search_term = ['politics','foreign']
url = f"{base_url}/search?q={search_term}&hl=en-US&gl=US&ceid=US:en"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Send a GET request to the Google News search results page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Find article elements
article_elements = soup.find_all("h3", class_="ipQwMb ekueJc RD0gLb")

# Create a list to store the extracted data
data = []

# Extract article titles, links, images, and dates and store in the data list
for article_element in article_elements:
    article_title = article_element.get_text()
    article_link = base_url + article_element.a["href"][1:]

    # Extract the publishing date if available
    date_element = article_element.find_next("time")
    if date_element and "datetime" in date_element.attrs:
        article_date = date_element["datetime"][:10]  # Extract the first 10 characters as the date
    else:
        article_date = "No Date Available"
    
    # Extract the image if available
    image_element = article_element.find_next("img")
    if image_element and "src" in image_element.attrs:
        article_image = image_element["src"]
    else:
        article_image = "No Image Available"
    
    data.append([article_title, article_link, article_date, article_image])

# Define the CSV file name
csv_file_name = "political_news1.csv"

# Write the data to the CSV file
with open(csv_file_name, "w", newline="", encoding="utf-8") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["Title", "Link", "Date", "Image"])  # Write header
    csv_writer.writerows(data)  # Write data rows

print(f"Political news data with images has been saved to {csv_file_name}")

In [ ]:
import pandas as pd
dfGS22 = pd.read_csv('political_news1.csv')
dfGS22

In [38]:
import pandas as pd
from IPython.display import HTML, display


# Display the DataFrame with images
def display_images(row):
    images_html = ""
    image_data_list = row["Image"].split(", ") if isinstance(row["Image"], str) else []

    for idx, image_data_encoded in enumerate(image_data_list):
        image_html = f'<img src="data:image/jpeg;base64,{image_data_encoded}" width="200" />'
        images_html += image_html

    return images_html

dfGS2["Image"] = dfGS2.apply(display_images, axis=1)
display(HTML(dfGS2.to_html(escape=False)))


,Title,Link,Date,Image
0,U.S-China Rivalry Plays Out in Vanuatu's Politics,https://news.google.com/articles/CBMiU2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyMy8wOC8yOC92YW51YXR1LXVzLWNoaW5hLWNvbXBldGl0aW9uLXBhY2lmaWMtc2VjdXJpdHkv0gEA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-28,""" width=""200"" />"
1,Georgetown University Institute of Politics and Public Service and ...,https://news.google.com/articles/CBMipwFodHRwczovL3d3dy5hcC5vcmcvcHJlc3MtcmVsZWFzZXMvMjAyMy9nZW9yZ2V0b3duLXVuaXZlcnNpdHktaW5zdGl0dXRlLW9mLXBvbGl0aWNzLWFuZC1wdWJsaWMtc2VydmljZS1hbmQtdGhlLWFzc29jaWF0ZWQtcHJlc3MtdG8taG9zdC1uYXRpb25hbC1zZWN1cml0eS1hbmQtZm9yZWlnbi1wb9IBAA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-28,""" width=""200"" />"
2,Libya’s Foreign Minister Najla al-Mangoush dismissed: Sources,https://news.google.com/articles/CBMic2h0dHBzOi8vd3d3LmFsamF6ZWVyYS5jb20vbmV3cy8yMDIzLzgvMjgvbGlieWEtcHJvdGVzdHMtYWZ0ZXItZm9yZWlnbi1taW5pc3Rlci1yZXBvcnRlZGx5LW1ldC13aXRoLWlzcmFlbGktb2ZmaWNpYWzSAXdodHRwczovL3d3dy5hbGphemVlcmEuY29tL2FtcC9uZXdzLzIwMjMvOC8yOC9saWJ5YS1wcm90ZXN0cy1hZnRlci1mb3JlaWduLW1pbmlzdGVyLXJlcG9ydGVkbHktbWV0LXdpdGgtaXNyYWVsaS1vZmZpY2lhbA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-28,""" width=""200"" />"
3,Vivek Ramaswamy unveils foreign policy agenda,https://news.google.com/articles/CBMiY2h0dHBzOi8vbnkxLmNvbS9ueWMvYWxsLWJvcm91Z2hzL3BvbGl0aWNzLzIwMjMvMDgvMjgvdml2ZWstcmFtYXN3YW15LXVudmVpbHMtZWNvbm9taWMtcG9saWN5LWFnZW5kYdIBAA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-28,""" width=""200"" />"
4,GOP Presidential Candidate Forum on National Security & Foreign ...,https://news.google.com/articles/CBMiR2h0dHBzOi8vcG9saXRpY3MuZ2VvcmdldG93bi5lZHUvZm9ydW0vZ29wLXByZXNpZGVudGlhbC1jYW5kaWRhdGUtZm9ydW0v0gEA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-28,""" width=""200"" />"
5,Exclusive: Smuggler with ties to ISIS helped migrants enter US from ...,https://news.google.com/articles/CBMiYWh0dHBzOi8vd3d3LmNubi5jb20vMjAyMy8wOC8yOS9wb2xpdGljcy9taWdyYW50cy11cy1zb3V0aGVybi1ib3JkZXItc211Z2dsZXItaXNpcy10aWVzL2luZGV4Lmh0bWzSAWVodHRwczovL2FtcC5jbm4uY29tL2Nubi8yMDIzLzA4LzI5L3BvbGl0aWNzL21pZ3JhbnRzLXVzLXNvdXRoZXJuLWJvcmRlci1zbXVnZ2xlci1pc2lzLXRpZXMvaW5kZXguaHRtbA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-29,""" width=""200"" />"
6,Rick Scott isn’t sold on reauthorizing powerful foreign surveillance tool,https://news.google.com/articles/CBMiSmh0dHBzOi8vd3d3Lm1pYW1paGVyYWxkLmNvbS9uZXdzL3BvbGl0aWNzLWdvdmVybm1lbnQvYXJ0aWNsZTI3ODYxNDIzOS5odG1s0gFKaHR0cHM6Ly9hbXAubWlhbWloZXJhbGQuY29tL25ld3MvcG9saXRpY3MtZ292ZXJubWVudC9hcnRpY2xlMjc4NjE0MjM5Lmh0bWw?hl=en-US&gl=US&ceid=US%3Aen,2023-08-29,""" width=""200"" />"
7,Imran Khan Is Just the Beginning of Pakistan's Democratic Woes,https://news.google.com/articles/CBMic2h0dHBzOi8vZm9yZWlnbnBvbGljeS5jb20vMjAyMy8wOC8yOS9pbXJhbi1raGFuLWNvbnZpY3Rpb24tcGFraXN0YW4tZGVtb2NyYWN5LXBvbGl0aWNzLXNoYXJpZi1lbGVjdGlvbnMtb3Bwb3NpdGlvbi_SAQA?hl=en-US&gl=US&ceid=US%3Aen,2023-08-29,""" width=""200"" />"
8,[All Politics is Global] China's Confucius Institutes to Face Restrictions in Germany Amid Security Concerns,https://news.google.com/articles/CBMihAFodHRwczovL2phcGFuLWZvcndhcmQuY29tL2FsbC1wb2xpdGljcy1pcy1nbG9iYWwtY2hpbmFzLWNvbmZ1Y2l1cy1pbnN0aXR1dGVzLXRvLWZhY2UtcmVzdHJpY3Rpb25zLWluLWdlcm1hbnktYW1pZC1zZWN1cml0eS1jb25jZXJucy_SAYgBaHR0cHM6Ly9qYXBhbi1mb3J3YXJkLmNvbS9hbGwtcG9saXRpY3MtaXMtZ2xvYmFsLWNoaW5hcy1jb25mdWNpdXMtaW5zdGl0dXRlcy10by1mYWNlLXJlc3RyaWN0aW9ucy1pbi1nZXJtYW55LWFtaWQtc2VjdXJpdHktY29uY2VybnMvYW1wLw?hl=en-US&gl=US&ceid=US%3Aen,2023-08-29,""" width=""200"" />"
9,James Cleverly heading to China to 'strengthen channels of communication',https://news.google.com/articles/CBMia2h0dHBzOi8vbmV3cy5za3kuY29tL3N0b3J5L2phbWVzLWNsZXZlcmx5LWhlYWRpbmctdG8tY2hpbmEtdG8tc3RyZW5ndGhlbi1jaGFubmVscy1vZi1jb21tdW5pY2F0aW9uLTEyOTUwMDYz0gFvaHR0cHM6Ly9uZXdzLnNreS5jb20vc3RvcnkvYW1wL2phbWVzLWNsZXZlcmx5LWhlYWRpbmctdG8tY2hpbmEtdG8tc3RyZW5ndGhlbi1jaGFubmVscy1vZi1jb21tdW5pY2F0aW9uLTEyOTUwMDYz?hl=en-US&gl=US&ceid=US%3Aen,2023-08-29,""" width=""200"" />"
